In [1]:
from imports import *

In [39]:
#helper function: for image processing
def resize(frame, scale=0.3):
    width = int(frame.shape[1] * scale)
    height = int(frame.shape[0] * scale)
    return cv2.resize(frame, (width,height), interpolation=cv2.INTER_AREA)
def final(frame):
    return cv2.resize(frame, (630,880), interpolation=cv2.INTER_AREA)
def wait():
    cv2.waitKey(0)
  #helper function:returns a string with only characters
def extract_characters(s):
    whitelist = r'[a-zA-Z0-9\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FFF]'
    return ''.join(re.findall(whitelist, s))
#helper function: returns a list with mercari products [name, price, availability]
def extract_item_details(output, text):
    results = []
    for line in output:
        # Check if every character in 'text' exists in the current line
        if all(char.lower() in line.lower() for char in text):
            item = {}
            
            # Check if the item is sold out
            if "売り切れ" in line:
                item['availability'] = "sold out"
            else:
                item['availability'] = "available"
                item['name'] = re.split(r"の画像", line)[0]
            item['product_name'] = re.sub('\u3000', ' ', re.split(r"の画像", line)[0])
            item['price'] = re.search(r"(\d+,?\d+)(円)", line).group(1)
            results.append(item)
    return results
#helper function: append df to excel
def to_excel(df, file_name):
 # Check if DataFrame is valid and non-empty
    if df is None or df.empty:
        print("Error: Invalid or empty DataFrame provided.")
        return

    try:
        # Check if file exists, read existing data, and append the new data
        if os.path.exists(file_name):
            existing_data = pd.read_excel(file_name)
            df = pd.concat([existing_data, df], ignore_index=True)

        # Write the DataFrame to Excel
        df.to_excel(file_name, index=False)
        #print(f"Data has been appended to '{file_name}'.")

    except PermissionError:
        print(f"Error: Permission denied. Can't write to '{file_name}'.")
    except FileNotFoundError:
        print(f"Error: Invalid path specified: '{file_name}'.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
#helper function: returns final price
def price(df, info, letter):
    try:
        # Ensure the DataFrame has the expected columns
        if not set(['price', 'availability']).issubset(df.columns):
            print(f"Error: DataFrame doesn't have the required columns for {info[1]}.")
            return -1

        # Safely convert price to integer
        df['price'] = df['price'].str.replace(',', '', regex=False).astype(int)

        # Calculate IQR for price, with checks for dataframe length
        if len(df) < 4:  # Arbitrary threshold; adjust as needed
            print(f"Warning: Results for {info[1]} has only {len(df)} row(s). Results might not be reliable.")

        Q1 = df['price'].quantile(0.25)
        Q3 = df['price'].quantile(0.75)
        IQR = Q3 - Q1
        median = df['price'].median()
        # Define bounds for outliers
        lower_bound = Q1 / 2.5
        upper_bound = Q3 * 3
        print(Q3)
        # Filter out outliers
        df_filtered = df[(df['price'] >= lower_bound) & (df['price'] <= upper_bound)]
        # Calculate average price for available items
        df_available = df_filtered[df_filtered['availability'] == 'available'].reset_index(drop=True)
        sorted_available = df_available.sort_values(by=['price'], ascending=True)
        if len(sorted_available) >= 2:
            available_price = sorted_available.iloc[:2]['price'].mean()
        elif len(sorted_available) == 1:
            available_price = sorted_available['price'].mean()
        else:
            available_price = 0

        # Calculate weighted average price for sold items
        df_sold = df_filtered[df_filtered['availability'] == 'sold out'].reset_index(drop=True)
        df_sold['recency_weight'] = df_sold.index[::-1] + 2
        numerator = (df_sold['price'] * df_sold['recency_weight']).sum()
        denominator = df_sold['recency_weight'].sum()
        #print(df_available, df_sold)
        # Check for division by zero
        if denominator == 0:
            sold_price = 0
        else:  
            sold_price = numerator / denominator
        #print(f"available: {available_price}, sold: {sold_price}")
        if available_price == 0:
            final_price = sold_price
        elif sold_price == 0:
            final_price = available_price
        else:
            final_price = (available_price + sold_price) / 2
        #print(f"Final price for {info[1]}: {final_price}")
        price_row = pd.DataFrame({'id': [info[0]], 'estimated_price': [final_price], 'name': [info[1]], 'number_of_items': [len(df)], 'sold_items': [len(df_sold)], 'available_items': [len(df_available)], 'grade': [letter]})
        to_excel(price_row, 'price.xlsx')
        return final_price

    except Exception as e:
        print(f"An error occurred for {info[1]}: {e}")
        return -1
#helper function: extract the id name from the string
def extract_id(s):
        pattern = r"[A-Za-z]+[A/]?[SW35][Ss\d]+-[Ss\d]+\s*(?:RR|SEC|SP|SSP|OFR|PR|HYR|RRR|R|SPM|5P|S5P)"
        matches = re.findall(pattern, s)[0]
        if matches[2] == 'A' and matches[3] in ['S', 'W','3', '5']:
            matches = matches[:2] + '/' + matches[3:]
        if matches[3] == 'A' and matches[4] in ['S', 'W','3', '5']:
            matches = matches[:3] + '/' + matches[4:]
        if matches[3] in ['3','5'] and matches[2] == '/':
            matches = matches[:3] + 'S' + matches[4:]
        if matches[4] in ['3','5'] and matches[3] == '/':
            matches = matches[:4] + 'S' + matches[5:]
        if matches[-2] == '5':
            matches = matches[:-2] + 'S' + matches[-1]
        if '/' not in matches:
        # Check if the fourth character is a digit
            if matches[3].isdigit():
                # Add '/' after the 2nd character
                matches = matches[:2] + '/' + matches[2:]
            else:
                # Add '/' after the 3rd character
                matches = matches[:3] + '/' + matches[3:]
        return matches
def resize(image_path):
    if not os.path.exists(image_path):
        print(f"Error: File '{image_path}' not found.")
        return -1
    # Preprocess image
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Couldn't read the image '{image_path}'.")
        return -1
    height, width, _ = img.shape

    # Crop the left quarter of the image
    left_quarter = img[int(height/2):height, 0:int(width/2)]

    # Resize the left quarter to original dimensions
    resized_left_quarter = cv2.resize(left_quarter, (width, height), interpolation=cv2.INTER_CUBIC)

    # Save the processed image
    tem_file = 'C:\\Coding\\python\\ws\\tem.jpg'
    cv2.imwrite(tem_file, resized_left_quarter)
    return tem_file

In [40]:
x = mercari(yuyu('BD/W54-056SP'),'A')

https://jp.mercari.com/search?keyword=SP%20%20真っ白な居場所%20白鷺千聖
1666.0


In [ ]:
x

In [3]:
#main function: for image processing. Returns the cropped image 
def crop_image(image_file):
    try:
        # Ensure the file exists
        if not os.path.exists(image_file):
            print(f"Error: File '{image_file}' not found.")
            return None

        # Preprocess image
        img = cv2.imread(image_file)
        if img is None:
            print(f"Error: Couldn't read the image '{image_file}'.")
            return None

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        egray = cv2.equalizeHist(gray)
        binary = cv2.adaptiveThreshold(egray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                       cv2.THRESH_BINARY_INV, 11, 2)
        blur = cv2.GaussianBlur(binary, (9, 9), cv2.BORDER_DEFAULT)
        canny = cv2.Canny(blur, 175, 250)
        dilated = cv2.dilate(canny, (11,11), iterations=3)

        # Create contours
        contours, hierarchies = cv2.findContours(dilated, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

        # Filter out small contours
        min_contour_area = 150
        filtered_contours = [contour for contour in contours if cv2.contourArea(contour) > min_contour_area]

        # Check if any valid contours were found
        if not filtered_contours:
            print("Error: No valid contours found.")
            return None

        # Merge all contour points
        merged_contours = np.vstack(filtered_contours)
        rect = cv2.minAreaRect(merged_contours)
        box = cv2.boxPoints(rect)
        box = np.intp(box)

        # Warp image
        box = sorted(box, key=lambda x: x[1])
        if box[0][0] > box[1][0]:
            box[0], box[1] = box[1], box[0]
        if box[2][0] > box[3][0]:
            box[2], box[3] = box[3], box[2]

        width = int(rect[1][0])
        height = int(rect[1][1])
        dst = np.array([
            [0, 0],
            [width-1, 0],
            [0, height-1],
            [width-1, height-1],
        ], dtype="float32")
        M = cv2.getPerspectiveTransform(np.array(box, dtype="float32"), dst)
        warped = cv2.warpPerspective(img, M, (width, height))
        f = final(warped)  

        # Preprocess cropped image
        x_start, y_start, x_end, y_end =  63, 770, 170, 788 
        roi = f[y_start:y_end, x_start:x_end]
        roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        roi_egray = cv2.equalizeHist(roi_gray)
        roi_resized = cv2.resize(roi_egray, None, fx=4, fy=4, interpolation=cv2.INTER_CUBIC)
        tem_file = 'tem.jpg'
        cv2.imwrite(tem_file, roi_resized)
        return tem_file

    except Exception as e:
        print(f"An error occurred: {e}")
        return None
#main function: for finding the text in the image. Returns the text
def read_image(image_path):
    try:
        # Check if chromedriver exists at the specified path
        service_path = r"C:/SeleniumDriver/chromedriver.exe"
        if not os.path.exists(service_path):
            print(f"Error: Chromedriver not found at '{service_path}'.")
            return -1

        # Check if the image exists
        final_path = 'C:\\Coding\\python\\ws\\' + image_path
        if not os.path.exists(final_path):
            print(f"Error: Image not found at '{final_path}'.")
            return -1

        # Initialize web driver
        service = Service(service_path)
        driver = webdriver.Chrome(service=service)
        driver.get('https://www.google.com/imghp')
        driver.implicitly_wait(1)
        # Interact with web elements using original logic
        camera_button = driver.find_element(By.CLASS_NAME, 'Gdd5U')
        camera_button.click()

        upload_button = driver.find_element(By.CLASS_NAME, 'DV7the')
        upload_button.click()
        
        time.sleep(0.5)
        pyautogui.write(resize(final_path))
        time.sleep(0.5)
        pyautogui.press('enter')
        time.sleep(0.5)
        pyautogui.press('enter')
        text_button = driver.find_element(By.XPATH, '//*[@id="ucj-4"]/span[1]')
        text_button.click()
        select_button = driver.find_element(By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div[2]/div/c-wiz/div/div[2]/c-wiz/div/div/div/div[2]/div[1]/div/div/div/div[2]/div/button/span')
        select_button.click()
        time.sleep(0.5)
        element = driver.find_element(By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div[2]/div/c-wiz/div/div[2]/c-wiz/div/div/span/div/div[2]')
        text = re.split(r'CH',element.text.upper())[1]
        print(text)
        return extract_id(text)
    except Exception as e:
        print(f"An error occurred: {e}")
        return -1

    finally:
        # Ensure the browser is closed at the end
        if 'driver' in locals() and driver:
            driver.close()
#main function: searching the details in yuyutei. Returns list of [id, name, yuyutei price] 0,1,2
def yuyu(text):
    try:
        no_spaces = re.sub(r'\s+', '', text)
        split = re.split(r'/', no_spaces)
        series = split[0]
        id_no = re.search(r'([A-Z]+\d+-\d+)', split[1]).group(0)
        rarity = re.search(r'[A-Z]+$', split[1]).group(0)
        rarity_name = '' if rarity == 'RR' else rarity
        yu_url = 'https://yuyu-tei.jp/game_ws/sell/sell_price.php?name='+series+'%2F'+ id_no+rarity_name+'&rare=&type&kizu=0'
        
        s = requests.Session()
        headers = {'accept-language': 'zh-CN,zh;q=0.9,en;q=0.8,zh-TW;q=0.7'}
        s.headers.update(headers)
        response = s.get(yu_url)
        
        # Check if the request was successful
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, 'lxml')
        card_list = soup.find("ul", class_="card_list")
        
        # Check if BeautifulSoup finds the expected elements
        if card_list:
            img_element = card_list.find("img")
            price_element = card_list.find("p", class_='price')
            
            if img_element and price_element:
                name = img_element["alt"]
                price = price_element.text
                price = re.findall("[0-9]*", price)[1]
                name = re.findall("\S[^(]*", name) 
                yuyu_name = re.sub('["“”#]',' ',name[0])
                #print(text, yuyu_name, price)
                return [text, yuyu_name, price]
            else:
                print(f"Couldn't find expected elements in {text}.")
                return -1
        else:
            print(f"Card list not found in {text}.")
            return -1

    except requests.RequestException as e:
        print(f"Request error for {text}: {e}")
        return -1
    except Exception as e:
        print(f"An error occurred for {text}: {e}")
        return -1
#main function: returns a dataframe with the following columns: id, name, price, availability
def mercari(info, letter):
    try:
        service = Service(r"C:/SeleniumDriver/chromedriver.exe")

        # Build the URL
        url = 'https://jp.mercari.com/search?keyword=' + re.sub(' ', '%20', info[1])
        print(url)
        # Initialize the WebDriver
        driver = webdriver.Chrome(service=service)
        driver.get(url)
        time.sleep(3)  # Adjust this sleep time if needed

        # Get page source and parse with BeautifulSoup
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        # Ensure driver is closed after extraction
        driver.quit()

        # Extract required elements
        div_elems = soup.find_all('div', class_='merItemThumbnail fluid__a6f874a2', limit=15)
        if not div_elems:
            print(f"Warning: No elements found on the Mercari page for {info[1]}. (1)")
            return -1

        elems = [elem['aria-label'] for elem in div_elems if 'aria-label' in elem.attrs]

        # Check if any elements were found
        if not elems:
            print(f"Warning: No elements found on the Mercari page for {info[1]}. (2)")
            return -1

        data = extract_item_details(elems, extract_characters(info[1]))
        if not data:
            print(f"Warning: No elements found on the Mercari page for {info[1]}. (3)")
            return -1
        for item in data:
            item['id'] = info[0]
            item['name'] = info[1]
        # Create DataFrame and save to Excel
        df = pd.DataFrame(data, columns=['id', 'name','product_name', 'price', 'availability'])
        to_excel(df, "raw.xlsx")
        price(df, info, letter)
        
        return df

    except Exception as e:
        print(f"An error occurred in the 'mercari' function for {info[1]}: {e}")
        return -1

In [96]:
read_image('img\\002_C.jpg')

 BDW63-058SPMB SPM “あたし POZIER 5000 OBANG DREAM! PROJECT CRAFT EGG


'BD/W63-058SP'

In [4]:
#integrates the main functions above, returns info = yuyu(text)
def collect_data(img, letter):
    text = read_image(img)
    if text == -1:
        print(f'{img} cannot be read')
        return
    info = yuyu(text)
    if text == -1:
        print(f'{text} cannot be found in yuyutei')
        return
    mercari(info, letter)
    print("\n")
    return info   

In [5]:
def add_ind_data(text, letter):
    info = yuyu(text)
    if text == -1:
        print(f'{text} cannot be found in yuyutei')
        return
    mercari(info, letter)
    return info   


In [6]:
def collect_all(x,y):
    all_files = os.listdir('img')
    for file in all_files:
        # Split the filename into its components
        name, ext = os.path.splitext(file)
        
        # Check if the file is a JPEG
        if ext.lower() == '.jpg':
            # Split the name into its numeric and letter components
            try:
                num, let = name.split('_')
                num = int(num)
            except ValueError:
                continue  # Skip files that don't match the expected format
            
            # Check if the file matches the given criteria
            if x <= num <= y and let in ['A', 'B', 'C', 'D', 'E', 'F']:
                collect_data('img\\'+file, let)

    # Call the abc function with the filtered files


In [97]:
collect_all(1,4)

BD/W95-100SP SP “未知と POZIER 4000 ANNI BANG DREAM! PROJECT CRAFT EGG
https://jp.mercari.com/search?keyword=SP%20%20未知との遭遇%20八潮瑠唯


 BDW63-058SPMB SPM “あたし POZIER 5000 OBANG DREAM! PROJECT CRAFT EGG
https://jp.mercari.com/search?keyword=SPM%20%20あたしはあたし%20氷川日菜


 BD/W95-045SP SP 66 10500 ANN BANG DREAM! PROJECT CRAFT EGG
https://jp.mercari.com/search?keyword=SP%20%20六花の全力%20ロック


 BDAW54-056SP SP ・真・ 5500 OBANG DREAM! PROJECT
https://jp.mercari.com/search?keyword=SP%20%20真っ白な居場所%20白鷺千聖




In [8]:
#add_ind_data('BD/W63-058SP', 'B')

In [95]:
os.remove('price.xlsx')
os.remove('raw.xlsx')